In [4]:
import sqlite3

'''
1. How to Create A Table?
>> CREATE TABLE Users(Name VARCHAR(128), Email VARCHAR(128))
>> CREATE TABLE IF NOT EXISTS Users(Name VARCHAR(128), Email VARCHAR(128))

2. How to Insert Data into Table?
>> INSERT INTO Users(Name, Email) VALUES('Kelly','kel@gmail.com')

3. How to Delete A Row?
>> DELETE FROM Users WHERE Email='ted@gmail.com'

4. How to Update A Entry?
>> UPDATE Users SET Name='Chris' WHERE Email='csev@gmail.com'
>> Explaination: "WHERE" Clause is Important. Because, Otherwise It'd do the changes for all the tables. Therefore, We must Narrow it down.
'''

print(f"SQLITE3 Installed Version: {sqlite3.version}")

SQLITE3 Installed Version: 2.6.0
